In [ ]:
%sql
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    afih.equnr AS equipamento,
    iloa.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
LEFT JOIN poseidon_uc.saps4hanarawview.afih AS afih
    ON afih.aufnr = aufk.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.iloan = afih.iloan
WHERE 
    aufk.auart IN ('GIA','GAA','GRA')
    AND SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
ORDER BY aufk.erdat DESC, aufk.aufnr
LIMIT 1000


In [ ]:
%sql
WITH orders_ok AS (
    SELECT DISTINCT afih.aufnr
    FROM poseidon_uc.saps4hanarawview.afih afih
    JOIN poseidon_uc.saps4hanarawview.iloa iloa
      ON iloa.iloan = afih.iloan
    WHERE SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
),
afih_pick AS (
    SELECT
        afih.aufnr,
        MAX(afih.equnr) AS equnr,
        MAX(afih.iloan) AS iloan
    FROM poseidon_uc.saps4hanarawview.afih afih
    WHERE afih.aufnr IN (SELECT aufnr FROM orders_ok)
    GROUP BY afih.aufnr
)
SELECT
    k.aufnr AS ordem,
    k.auart AS tipo_ordem,
    k.ktext AS texto_breve,
    k.erdat AS data_criacao,
    k.bukrs AS empresa,
    k.werks AS centro_trabalho,
    p.equnr AS equipamento,
    i.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk k
JOIN orders_ok ok ON ok.aufnr = k.aufnr
LEFT JOIN afih_pick p ON p.aufnr = k.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa i ON i.iloan = p.iloan
WHERE 
    k.auart IN ('GIA','GAA','GRA')
ORDER BY k.erdat DESC, k.aufnr
LIMIT 1000



In [ ]:
%sql
-- Extração de Ordens de Manutenção (similar à IW38)
-- Filtra por tipo de ordem (GIA, GAA, GRA) e local de instalação (GHIN, GHEN, GHSM)
-- Garante uma linha por ordem (aufnr único)

WITH ordens_validas AS (
    -- Identifica ordens que possuem locais de instalação válidos
    SELECT DISTINCT afih.aufnr
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    INNER JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
        ON iloa.iloan = afih.iloan
    WHERE SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN', 'GHEN', 'GHSM')
),
dados_afih AS (
    -- Seleciona dados principais do AFIH (1 registro por ordem)
    -- Em caso de múltiplos registros, pega o mais recente/relevante
    SELECT
        afih.aufnr,
        MAX(afih.equnr) AS equipamento,
        MAX(afih.iloan) AS iloan,
        MAX(afih.iwerk) AS planta_manutencao,
        MAX(afih.ingpr) AS grupo_planejador,
        MAX(afih.ilart) AS tipo_atividade,
        MAX(afih.qmnum) AS numero_notificacao
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.aufnr IN (SELECT aufnr FROM ordens_validas)
    GROUP BY afih.aufnr
)
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.ernam AS criado_por,
    aufk.aedat AS data_alteracao,
    aufk.aenam AS alterado_por,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    aufk.kostl AS centro_custo,
    aufk.waers AS moeda,
    aufk.loekz AS ind_exclusao,
    afih.equipamento,
    iloa.tplnr AS local_instalacao,
    iloa.stort AS desc_local_instalacao,
    afih.planta_manutencao,
    afih.grupo_planejador,
    afih.tipo_atividade,
    afih.numero_notificacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
INNER JOIN ordens_validas AS ov
    ON ov.aufnr = aufk.aufnr
LEFT JOIN dados_afih AS afih
    ON afih.aufnr = aufk.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.iloan = afih.iloan
WHERE 
    aufk.auart IN ('GIA', 'GAA', 'GRA')  -- Filtra tipos de ordem
ORDER BY aufk.erdat DESC, aufk.aufnr
LIMIT 1000

In [ ]:
%sql
-- Extração COMPLETA da tabela AUFK (todos os campos)
-- Para análise detalhada dos registros duplicados
SELECT
    aufk.DI_SEQUENCE_NUMBER,
    aufk.DI_OPERATION_TYPE,
    aufk.MANDT AS cliente_sap,
    aufk.AUFNR AS ordem,
    aufk.AUART AS tipo_ordem,
    aufk.AUTYP AS tipo_automatico,
    aufk.REFNR AS numero_referencia,
    aufk.ERNAM AS usuario_criacao,
    aufk.ERDAT AS data_criacao,
    aufk.AENAM AS usuario_alteracao,
    aufk.AEDAT AS data_alteracao,
    aufk.KTEXT AS texto_breve,
    aufk.LTEXT AS texto_longo,
    aufk.BUKRS AS empresa,
    aufk.WERKS AS centro_trabalho,
    aufk.GSBER AS area_negocio,
    aufk.KOKRS AS area_controle,
    aufk.CCKEY AS chave_centro_custo,
    aufk.KOSTV AS centro_custo_verificacao,
    aufk.STORT AS local_armazenagem,
    aufk.SOWRK AS centro_trabalho_origem,
    aufk.ASTKZ AS indicador_estoque,
    aufk.WAERS AS moeda,
    aufk.ASTNR AS numero_estoque,
    aufk.STDAT AS data_inicio,
    aufk.ESTNR AS numero_estimativa,
    aufk.PHAS0 AS fase_0,
    aufk.PHAS1 AS fase_1,
    aufk.PHAS2 AS fase_2,
    aufk.PHAS3 AS fase_3,
    aufk.PDAT1 AS data_planejada_1,
    aufk.PDAT2 AS data_planejada_2,
    aufk.PDAT3 AS data_planejada_3,
    aufk.IDAT1 AS data_inicio_1,
    aufk.IDAT2 AS data_inicio_2,
    aufk.IDAT3 AS data_inicio_3,
    aufk.OBJID AS id_objeto,
    aufk.VOGRP AS grupo_venda,
    aufk.LOEKZ AS indicador_exclusao,
    aufk.PLGKZ AS indicador_planejamento,
    aufk.KVEWE AS area_uso,
    aufk.KAPPL AS aplicacao,
    aufk.KALSM AS esquema_calculo,
    aufk.ZSCHL AS chave_calculo,
    aufk.ABKRS AS area_abono,
    aufk.KSTAR AS elemento_custo,
    aufk.KOSTL AS centro_custo,
    aufk.SAKNR AS conta_contabil,
    aufk.SETNM AS nome_conjunto,
    aufk.CYCLE AS ciclo,
    aufk.SDATE AS data_inicio_ciclo,
    aufk.SEQNR AS numero_sequencia,
    aufk.USER0 AS campo_usuario_0,
    aufk.USER1 AS campo_usuario_1,
    aufk.USER2 AS campo_usuario_2,
    aufk.USER3 AS campo_usuario_3,
    aufk.USER4 AS campo_usuario_4,
    aufk.USER5 AS campo_usuario_5,
    aufk.USER6 AS campo_usuario_6,
    aufk.USER7 AS campo_usuario_7,
    aufk.USER8 AS campo_usuario_8,
    aufk.USER9 AS campo_usuario_9,
    aufk.OBJNR AS numero_objeto_status,
    aufk.PRCTR AS centro_lucro,
    aufk.PSPEL AS elemento_wbs,
    aufk.AWSLS AS esquema_avaliacao,
    aufk.ABGSL AS esquema_abono,
    aufk.EB_POST AS indicador_posting,
    aufk.TXJCD AS codigo_jurisdicao,
    aufk.FUNC_AREA AS area_funcional,
    aufk.SCOPE AS escopo,
    aufk.PLINT AS integracao_planejamento,
    aufk.KDAUF AS ordem_venda,
    aufk.KDPOS AS posicao_venda,
    aufk.AUFEX AS ordem_externa,
    aufk.IVPRO AS processo_inventario,
    aufk.LOGSYSTEM AS sistema_logico,
    aufk.FLG_MLTPS AS flag_multiplos_tipos,
    aufk.ABUKR AS empresa_abono,
    aufk.AKSTL AS centro_custo_atividade,
    aufk.SIZECL AS classe_tamanho,
    aufk.IZWEK AS proposito_investimento,
    aufk.UMWKZ AS indicador_manutencao,
    aufk.KSTEMPF AS template_custo,
    aufk.ZSCHM AS esquema_calculo_z,
    aufk.PKOSA AS elemento_custo_principal,
    aufk.ANFAUFNR AS ordem_anterior,
    aufk.PROCNR AS numero_processo,
    aufk.PROTY AS tipo_processo,
    aufk.RSORD AS ordem_reserva,
    aufk.BEMOT AS motivo_requisicao,
    aufk.ADRNRA AS endereco_requisicao,
    aufk.ERFZEIT AS hora_criacao,
    aufk.AEZEIT AS hora_alteracao,
    aufk.CSTG_VRNT AS variante_custeio,
    aufk.COSTESTNR AS numero_estimativa_custo,
    aufk.VERAA_USER AS usuario_verificacao,
    aufk.ZCOD_PROJ AS codigo_projeto,
    aufk.ZCOD_CONTR AS codigo_contrato,
    aufk.ZCOD_MOTIV AS codigo_motivo,
    aufk.ZCOD_TEMAT AS codigo_tematico,
    aufk.ZCOD_EFEITO AS codigo_efeito,
    aufk.ZCOD_VALOR AS codigo_valor,
    aufk.ZCOD_TP_CO AS tipo_codigo_co,
    aufk.ZCOD_TP_EN AS tipo_codigo_en,
    aufk.ZCOD_TP_AL AS tipo_codigo_al,
    aufk.ZCOD_OBS_TX AS observacao_texto,
    aufk.ZINTEG_SIT AS situacao_integracao,
    aufk.ZPROCES_SIT AS situacao_processo,
    aufk.ZZEMPREEND1 AS empreendimento_1,
    aufk.ZZATO_AUT1 AS ato_autorizacao_1,
    aufk.ZZFUNCAO_TRA1 AS funcao_trabalho_1,
    aufk.ZZCOD_MOD1 AS codigo_modelo_1,
    aufk.ZZCOD_RAP1 AS codigo_rap_1,
    aufk.ZZCOD_CONTRATO1 AS codigo_contrato_1,
    aufk.EEW_AUFK_PS_DUMMY AS dummy_eew_ps,
    aufk.VNAME AS nome_variante,
    aufk.RECID AS id_registro,
    aufk.ETYPE AS tipo_entidade,
    aufk.OTYPE AS tipo_objeto,
    aufk.JV_JIBCL AS classe_jv,
    aufk.JV_JIBSA AS area_jv,
    aufk.JV_OCO AS oco_jv,
    aufk.CPD_UPDAT AS atualizacao_cpd,
    aufk.AD01PROFNR AS numero_perfil_ad01,
    aufk.VAPLZ AS codigo_postal_venda,
    aufk.WAWRK AS centro_trabalho_venda,
    aufk.FERC_IND AS indicador_ferc,
    aufk.AUFK_STATUS AS status_ordem,
    aufk.CLAIM_CONTROL AS controle_reivindicacao,
    aufk.UPDATE_NEEDED AS atualizacao_necessaria,
    aufk.UPDATE_CONTROL AS controle_atualizacao,
    aufk.ODQ_CHANGEMODE AS modo_mudanca_odq,
    aufk.ODQ_ENTITYCNTR AS contador_entidade_odq,
    aufk.current_ts AS timestamp_atual,
    aufk.yearCurrentTs AS ano_timestamp,
    aufk.monthCurrentTs AS mes_timestamp,
    aufk.dayCurrentTs AS dia_timestamp,
    aufk.dataImportacao AS data_importacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
WHERE
    -- YEAR(aufk.ERDAT) = 2025 AND
    aufk.AUART IN ('GIA','GAA','GRA') AND
    -- aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI') AND
    aufk.BUKRS IN ('GHIN') AND
    aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(aufk2.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS aufk2
        WHERE aufk2.AUFNR = aufk.AUFNR
    )
ORDER BY aufk.AUFNR, aufk.DI_SEQUENCE_NUMBER


In [ ]:
%sql
SELECT
    aufk.AUFNR AS ordem,
    aufk.KTEXT AS descricao,
    aufk.ERDAT AS dt_data_ordem, 
    aufk.ERFZEIT hr_data_ordem,
    aufk.AUART AS tipo_ordem,
    aufk.AEDAT AS dt_data_ref_etl 
FROM poseidon_uc.sapep1rawview.aufk aufk
WHERE
    aufk.AUART IN ('GAA', 'GIA', 'GRA', 'GOA')
ORDER BY aufk.AUFNR

In [ ]:
%sql
-- Extração ENRIQUECIDA com todas as tabelas complementares
-- Mostra todos os campos disponíveis de cada tabela relacionada 

SELECT
    -- CAMPOS DA TABELA AUFK (Ordens)
    aufk.AUFNR AS ordem,
    aufk.AUART AS tipo_ordem,
    aufk.BUKRS AS empresa,
    aufk.KTEXT AS texto_breve,
    
    -- CAMPOS DA TABELA AFKO (Cabeçalho Ordem Produção)
    afko.DI_SEQUENCE_NUMBER AS afko_seq,
    afko.DI_OPERATION_TYPE AS afko_operacao,
    afko.MANDT AS afko_cliente,
    afko.AUFNR AS afko_ordem,
    afko.GLTRP AS data_fim_programada,
    afko.GSTRP AS data_inicio_programada,
    afko.FTRMS AS fim_real_sistema,
    afko.GLTRS AS fim_real_global,
    afko.GSTRS AS inicio_real_sistema,
    afko.GSTRI AS data_inicio_real,
    afko.GETRI AS fim_inicio_real,
    afko.GLTRI AS data_fim_real,
    afko.FTRMI AS fim_real_inicio,
    afko.FTRMP AS fim_real_planejado,
    afko.RSNUM AS numero_roteiro,
    afko.GASMG AS quantidade_total,
    afko.GAMNG AS quantidade_ordem,
    afko.GMEIN AS unidade_medida,
    afko.PLNBEZ AS descricao_plano,
    afko.PLNTY AS tipo_plano,
    afko.PLNNR AS numero_plano,
    afko.PLNAW AS area_plano,
    afko.PLNAL AS alternativa_plano,
    afko.PVERW AS versao_plano,
    afko.PLAUF AS ordem_plano,
    afko.PLSVB AS versao_plano_sistema,
    afko.PLNME AS nome_plano,
    afko.PLSVN AS versao_plano_sistema_n,
    afko.PDATV AS data_versao_plano,
    afko.PAENR AS numero_alternativa_plano,
    afko.PLGRP AS grupo_planejador,
    afko.LODIV AS divisao_local,
    afko.STLTY AS tipo_lista_materiais,
    afko.STLBEZ AS descricao_lista_materiais,
    afko.STLST AS status_lista_materiais,
    afko.STLNR AS numero_lista_materiais,
    afko.SDATV AS data_versao_lista_materiais,
    afko.SBMNG AS quantidade_base,
    afko.SBMEH AS unidade_medida_base,
    afko.SAENR AS numero_alternativa_lista,
    afko.STLAL AS alternativa_lista_materiais,
    afko.STLAN AS uso_lista_materiais,
    afko.SLSVN AS versao_lista_materiais_sistema,
    afko.SLSBS AS versao_lista_materiais_sistema_b,
    afko.AUFLD AS campo_ordem,
    afko.DISPO AS responsavel_mrp,
    afko.AUFPL AS numero_plano_ordem,
    afko.FEVOR AS centro_planejamento,
    afko.FHORI AS horizonte_planejamento,
    afko.TERKZ AS indicador_termino,
    afko.REDKZ AS indicador_reducao,
    afko.APRIO AS prioridade_ordem,
    afko.NTZUE AS numero_termino_zue,
    afko.VORUE AS precedencia,
    afko.PROFID AS id_perfil,
    afko.VORGZ AS grupo_operacao,
    afko.SICHZ AS grupo_seguranca,
    afko.FREIZ AS grupo_liberacao,
    afko.UPTER AS indicador_atualizacao,
    afko.BEDID AS id_necessidade,
    afko.PRONR AS numero_projeto,
    afko.ZAEHL AS contador,
    afko.MZAEHL AS contador_m,
    afko.ZKRIZ AS indicador_critico,
    afko.PRUEFLOS AS numero_lote_inspecao,
    afko.KLVARP AS variavel_calculo,
    afko.KLVARI AS variavel_calculo_i,
    afko.RGEKZ AS indicador_regra,
    afko.PLART AS tipo_plano_art,
    afko.FLG_AOB AS flag_aob,
    afko.FLG_ARBEI AS flag_arbei,
    afko.GLTPP AS fim_planejado_pp,
    afko.GSTPP AS inicio_planejado_pp,
    afko.GLTPS AS fim_planejado_ps,
    afko.GSTPS AS inicio_planejado_ps,
    afko.FTRPS AS fim_real_ps,
    afko.RDKZP AS indicador_reducao_pp,
    afko.TRKZP AS indicador_termino_pp,
    afko.RUECK AS numero_confirmacao,
    afko.RMZHL AS contador_confirmacao,
    afko.IGMNG AS quantidade_ignorada,
    afko.RATID AS id_razao,
    afko.GROID AS id_grupo,
    afko.CUOBJ AS objeto_configuracao,
    afko.GLUZS AS fim_utc_sistema,
    afko.GSUZS AS inicio_utc_sistema,
    afko.REVLV AS nivel_revisao,
    afko.RSHTY AS tipo_estrutura_roteiro,
    afko.RSHID AS id_estrutura_roteiro,
    afko.RSNTY AS tipo_roteiro_sistema,
    afko.RSNID AS id_roteiro_sistema,
    afko.NAUTERM AS termino_automatico,
    afko.NAUCOST AS custo_automatico,
    afko.STUFE AS nivel,
    afko.WEGXX AS caminho_xx,
    afko.VWEGX AS caminho_v,
    afko.ARSNR AS numero_artigo,
    afko.ARSPS AS posicao_artigo,
    afko.MAUFNR AS ordem_principal,
    afko.LKNOT AS no_esquerdo,
    afko.RKNOT AS no_direito,
    afko.PRODNET AS rede_producao,
    afko.IASMG AS quantidade_ia,
    afko.ABARB AS trabalho_ab,
    afko.AUFNT AS tipo_ordem_auf,
    afko.AUFPT AS tipo_posicao_auf,
    afko.APLZT AS tempo_aplicacao,
    afko.NO_DISP AS sem_disponibilidade,
    afko.CSPLIT AS divisao_cs,
    afko.AENNR AS numero_alternativa,
    afko.CY_SEQNR AS numero_sequencia_cy,
    afko.BREAKS AS quebras,
    afko.VORGZ_TRM AS grupo_operacao_trm,
    afko.SICHZ_TRM AS grupo_seguranca_trm,
    afko.TRMDT AS data_trm,
    afko.GLUZP AS fim_utc_pp,
    afko.GSUZP AS inicio_utc_pp,
    afko.GSUZI AS inicio_utc_i,
    afko.GEUZI AS fim_utc_i,
    afko.GLUPP AS fim_utc_pp_2,
    afko.GSUPP AS inicio_utc_pp_2,
    afko.GLUPS AS fim_utc_ps,
    afko.GSUPS AS inicio_utc_ps,
    afko.CHSCH AS esquema_calculo_ch,
    afko.KAPT_VORGZ AS grupo_operacao_kapt,
    afko.KAPT_SICHZ AS grupo_seguranca_kapt,
    afko.LEAD_AUFNR AS ordem_lider,
    afko.PNETSTARTD AS data_inicio_rede_p,
    afko.PNETSTARTT AS tempo_inicio_rede_p,
    afko.PNETENDD AS data_fim_rede_p,
    afko.PNETENDT AS tempo_fim_rede_p,
    afko.KBED AS area_necessidade,
    afko.KKALKR AS calculadora_k,
    afko.SFCPF AS perfil_sfc,
    afko.RMNGA AS quantidade_rm,
    afko.GSBTR AS valor_gs,
    afko.VFMNG AS quantidade_vf,
    afko.NOPCOST AS sem_custo,
    afko.NETZKONT AS controle_rede,
    afko.ATRKZ AS indicador_atr,
    afko.OBJTYPE AS tipo_objeto,
    afko.CH_PROC AS processo_ch,
    afko.KAPVERSA AS versao_capacidade,
    afko.COLORDPROC AS processo_ordem_cor,
    afko.KZERB AS indicador_zerb,
    afko.CONF_KEY AS chave_confirmacao,
    afko.ST_ARBID AS id_trabalho_st,
    afko.VSNMR_V AS numero_versao_vs,
    afko.TERHW AS termino_hw,
    afko.SPLSTAT AS status_divisao,
    afko.COSTUPD AS atualizacao_custo,
    afko.MAX_GAMNG AS quantidade_maxima,
    afko.MES_ROUTINGID AS id_roteiro_mes,
    afko.TL_VERSN AS versao_tl,
    afko.BOM_VERSN AS versao_bom,
    afko.FLEXIBLE_PROCESSING AS processamento_flexivel,
    afko.ADPSP AS adpsp,
    afko.RMANR AS numero_rm,
    afko.POSNR_RMA AS posicao_rma,
    afko.POSNV_RMA AS posicao_nv_rma,
    afko.CFB_MAXLZ AS cfb_maxlz,
    afko.CFB_LZEIH AS cfb_lzeih,
    afko.CFB_ADTDAYS AS cfb_adtdays,
    afko.CFB_DATOFM AS cfb_datofm,
    afko.CFB_BBDPI AS cfb_bbdpi,
    afko.OIHANTYP AS tipo_oihan,
    afko.FSH_MPROD_ORD AS ordem_producao_fsh,
    afko.FLG_BUNDLE AS flag_bundle,
    afko.MILL_RATIO AS proporcao_mill,
    afko.BMEINS AS unidade_medida_bm,
    afko.BMENGE AS quantidade_bm,
    afko.MILL_OC_ZUSKZ AS mill_oc_zuskz,
    afko.dataImportacao AS afko_data_importacao,
    
    -- CAMPOS DA TABELA AFIH (Cabeçalho Ordem Manutenção)
    afih.DI_SEQUENCE_NUMBER AS afih_seq,
    afih.DI_OPERATION_TYPE AS afih_operacao,
    afih.MANDT AS afih_cliente,
    afih.AUFNR AS afih_ordem,
    afih.ARTPR AS categoria_prioridade,
    afih.PRIOK AS codigo_prioridade,
    afih.EQUNR AS equipamento,
    afih.BAUTL AS bauth,
    afih.ILOAN AS local_instalacao_funcional,
    afih.ILOAI AS iloai,
    afih.ANLZU AS anlzu,
    afih.IWERK AS planta_manutencao,
    afih.INGPR AS grupo_planejador_responsavel,
    afih.APGRP AS apgrp,
    afih.PM_OBJTY AS tipo_objeto_pm,
    afih.GEWRK AS centro_trabalho_principal,
    afih.KUNUM AS kunum,
    afih.ANING AS aning,
    afih.GAUZT AS gauzt,
    afih.GAUEH AS gaueh,
    afih.ANLBD AS anlbd,
    afih.ANLVD AS anlvd,
    afih.ANLBZ AS anlbz,
    afih.ANLVZ AS anlvz,
    afih.INSPK AS inspk,
    afih.DATAN AS datan,
    afih.WARPL AS plano_manutencao_associado,
    afih.ABNUM AS item_plano_manutencao,
    afih.WAPOS AS wapos,
    afih.LAUFN AS laufn,
    afih.OBKNR AS obknr,
    afih.REVNR AS revnr,
    afih.ADDAT AS addat,
    afih.ADUHR AS aduhr,
    afih.IPHAS AS iphas,
    afih.ILART AS tipo_atividade_manutencao,
    afih.QMNUM AS numero_notificacao_vinculada,
    afih.HISDA AS hisda,
    afih.AKKNZ AS akknz,
    afih.PLKNZ AS plknz,
    afih.SERIALNR AS numero_serie,
    afih.SERMAT AS material_serie,
    afih.DEVICEID AS id_dispositivo,
    afih.SCREENTY AS tipo_tela,
    afih.ADPSP AS adpsp_afih,
    afih.RSUPG AS rsupg,
    afih.CHANGEDDATETIME AS data_hora_alteracao,
    afih.MAINTORDPERSONRESPONSIBLE AS pessoa_responsavel_ordem_manutencao,
    afih.MAINTORDOVRLPROCPHASE AS fase_processo_sobreposicao_ordem_manutencao,
    afih.MAINTORDOVRLPROCSUBPHASE AS subfase_processo_sobreposicao_ordem_manutencao,
    afih.DUMMY_AFIH_INCL_EEW_PS AS dummy_afih_incl_eew_ps,
    afih./ISDFPS/OBJNR AS objnr_isdfps,
    afih./ISDFPS/MEQUI AS mequi_isdfps,
    afih./ISDFPS/MHIO_ADDATE AS mhio_addate_isdfps,
    afih./ISDFPS/MHIO_ADTIME AS mhio_adtime_isdfps,
    afih./ISDFPS/USERMODE AS usermode_isdfps,
    afih.UII AS uii,
    afih.DATAMODELVERSION AS versao_modelo_dados,
    afih.SERVICE_DOC_TYPE AS tipo_documento_servico,
    afih.SERVICE_DOC_ID AS id_documento_servico,
    afih.SERVICE_DOC_ITEM_ID AS id_item_documento_servico,
    afih.IS_BILLABLE AS faturável,
    afih.FLDLOGSDELIVISHELDONSHORE AS fldlogsdeliisheldonshore,
    afih.MAINTORDPROCMTINTEGSCENARIO AS cenario_integracao_mt_processo_ordem_manutencao,
    afih.ORCOD AS orcod,
    afih.PRMAN AS prman,
    afih.PRVAL AS prval,
    afih.TWTDE AS twtde,
    afih.MSTCK AS mstck,
    afih.LACD_DATE AS data_lacd,
    afih.OLD_LACD_DATE AS data_lacd_antiga,
    afih.ODQ_CHANGEMODE AS modo_mudanca_odq_afih,
    afih.ODQ_ENTITYCNTR AS contador_entidade_odq_afih,
    afih.dataImportacao AS afih_data_importacao,
    
    -- CAMPOS DA TABELA AFPO (Itens Ordem Produção)
    afpo.DI_SEQUENCE_NUMBER AS afpo_seq,
    afpo.DI_OPERATION_TYPE AS afpo_operacao,
    afpo.MANDT AS afpo_cliente,
    afpo.AUFNR AS afpo_ordem,
    afpo.POSNR AS posicao,
    afpo.PSOBS AS psobs,
    afpo.QUNUM AS qunum,
    afpo.QUPOS AS qupos,
    afpo.PROJN AS projn,
    afpo.PLNUM AS plnum,
    afpo.STRMP AS inicio_planejado,
    afpo.ETRMP AS fim_planejado,
    afpo.KDAUF AS ordem_venda_afpo,
    afpo.KDPOS AS posicao_venda_afpo,
    afpo.KDEIN AS kdein,
    afpo.BESKZ AS tipo_abastecimento,
    afpo.PSAMG AS quantidade_ps,
    afpo.PSMNG AS quantidade_psm,
    afpo.WEMNG AS quantidade_we,
    afpo.IAMNG AS quantidade_ia_afpo,
    afpo.AMEIN AS unidade_medida_am,
    afpo.MEINS AS unidade_medida_afpo,
    afpo.MATNR AS material,
    afpo.PAMNG AS quantidade_pa,
    afpo.PGMNG AS quantidade_pg,
    afpo.KNTTP AS categoria_item,
    afpo.TPAUF AS tpauf,
    afpo.LTRMI AS fim_real,
    afpo.LTRMP AS fim_planejado_ltr,
    afpo.KALNR AS numero_calculo,
    afpo.UEBTO AS uebto,
    afpo.UEBTK AS uebtk,
    afpo.UNTTO AS untto,
    afpo.INSMK AS insmk,
    afpo.WEPOS AS wepos,
    afpo.BWTAR AS tipo_avaliacao,
    afpo.BWTTY AS tipo_avaliacao_bw,
    afpo.PWERK AS centro_producao,
    afpo.LGORT AS deposito,
    afpo.UMREZ AS fator_conversao_um,
    afpo.UMREN AS denominador_conversao_um,
    afpo.WEBAZ AS tempo_armazenagem,
    afpo.ELIKZ AS indicador_liberacao,
    afpo.SAFNR AS safnr,
    afpo.VERID AS versao_producao,
    afpo.SERNR AS numero_serie_afpo,
    afpo.TECHS AS tecnicos,
    afpo.DWERK AS centro_trabalho_d,
    afpo.DAUTY AS tipo_autorizacao_d,
    afpo.DAUAT AS autorizacao_d,
    afpo.DGLTP AS fim_planejado_d,
    afpo.DGLTS AS fim_sistema_d,
    afpo.DFREI AS dfrei,
    afpo.DNREL AS dnrel,
    afpo.VERTO AS verto,
    afpo.SOBKZ AS sobkz,
    afpo.KZVBR AS kzvbr,
    afpo.WEWRT AS valor_we,
    afpo.WEUNB AS weunb,
    afpo.ABLAD AS local_descarga,
    afpo.WEMPF AS destinatario,
    afpo.CHARG AS lote,
    afpo.GSBER AS area_negocio_afpo,
    afpo.WEAED AS weaed,
    afpo.CUOBJ AS objeto_configuracao_afpo,
    afpo.KBNKZ AS kbnkz,
    afpo.ARSNR AS numero_artigo_afpo,
    afpo.ARSPS AS posicao_artigo_afpo,
    afpo.KRSNR AS numero_krs,
    afpo.KRSPS AS posicao_krs,
    afpo.KCKEY AS chave_kc,
    afpo.RTP01 AS rtp01,
    afpo.RTP02 AS rtp02,
    afpo.RTP03 AS rtp03,
    afpo.RTP04 AS rtp04,
    afpo.KSVON AS ksvon,
    afpo.KSBIS AS ksbis,
    afpo.OBJNP AS objnp,
    afpo.NDISR AS ndisr,
    afpo.VFMNG AS quantidade_vf_afpo,
    afpo.GSBTR AS valor_gs_afpo,
    afpo.KZAVC AS kzavc,
    afpo.KZBWS AS kzbws,
    afpo.XLOEK AS indicador_exclusao_afpo,
    afpo.SERNP AS sernp,
    afpo.ANZSN AS anzsn,
    afpo.OBJTYPE AS tipo_objeto_afpo,
    afpo.CH_PROC AS processo_ch_afpo,
    afpo.FXPRU AS fxpru,
    afpo.CUOBJ_ROOT AS objeto_configuracao_raiz,
    afpo.BERID AS area_requisicao,
    afpo.TECHS_COPY AS tecnicos_copia,
    afpo.SGT_SCAT AS sgt_scat,
    afpo.KUNNR2 AS cliente_2,
    afpo.FSH_SEASON_YEAR AS ano_temporada_fsh,
    afpo.FSH_SEASON AS temporada_fsh,
    afpo.FSH_COLLECTION AS colecao_fsh,
    afpo.FSH_THEME AS tema_fsh,
    afpo.FSH_SALLOC_QTY AS quantidade_alocacao_fsh,
    afpo.MILL_OC_AUFNR_U AS mill_oc_aufnr_u,
    afpo.MILL_OC_RUMNG AS mill_oc_rumng,
    afpo.MILL_OC_SORT AS mill_oc_sort,
    afpo.dataImportacao AS afpo_data_importacao,
    
    -- CAMPOS DA TABELA AFRU (Confirmações)
    afru.DI_SEQUENCE_NUMBER AS afru_seq,
    afru.DI_OPERATION_TYPE AS afru_operacao,
    afru.MANDT AS afru_cliente,
    afru.RUECK AS numero_confirmacao_afru,
    afru.RMZHL AS contador_confirmacao_afru,
    afru.ERSDA AS data_entrada,
    afru.ERZET AS tempo_entrada,
    afru.ERFMG AS quantidade_entrada,
    afru.ERFME AS unidade_medida_entrada,
    afru.ERFGE AS quantidade_entrada_erf,
    afru.ERFGR AS grupo_entrada,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada,
    afru.ERFGR_MG AS quantidade_grupo_entrada,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_2,
    afru.ERFGR_MG AS quantidade_grupo_entrada_2,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_2,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_3,
    afru.ERFGR_MG AS quantidade_grupo_entrada_3,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_3,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_4,
    afru.ERFGR_MG AS quantidade_grupo_entrada_4,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_4,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_5,
    afru.ERFGR_MG AS quantidade_grupo_entrada_5,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_5,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_6,
    afru.ERFGR_MG AS quantidade_grupo_entrada_6,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_6,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_7,
    afru.ERFGR_MG AS quantidade_grupo_entrada_7,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_7,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_8,
    afru.ERFGR_MG AS quantidade_grupo_entrada_8,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_8,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_9,
    afru.ERFGR_MG AS quantidade_grupo_entrada_9,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_9,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_10,
    afru.ERFGR_MG AS quantidade_grupo_entrada_10,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_10,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_11,
    afru.ERFGR_MG AS quantidade_grupo_entrada_11,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_11,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_12,
    afru.ERFGR_MG AS quantidade_grupo_entrada_12,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_12,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_13,
    afru.ERFGR_MG AS quantidade_grupo_entrada_13,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_13,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_14,
    afru.ERFGR_MG AS quantidade_grupo_entrada_14,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_14,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_15,
    afru.ERFGR_MG AS quantidade_grupo_entrada_15,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_15,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_16,
    afru.ERFGR_MG AS quantidade_grupo_entrada_16,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_16,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_17,
    afru.ERFGR_MG AS quantidade_grupo_entrada_17,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_17,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_18,
    afru.ERFGR_MG AS quantidade_grupo_entrada_18,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_18,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_19,
    afru.ERFGR_MG AS quantidade_grupo_entrada_19,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_19,
    afru.ERFGR_ME AS unidade_medida_grupo_entrada_20,
    afru.ERFGR_MG AS quantidade_grupo_entrada_20,
    afru.ERFGR_GE AS quantidade_grupo_entrada_erf_20,
    afru.dataImportacao AS afru_data_importacao,
    
    -- CAMPOS DA TABELA QMEL (Notificações Qualidade)
    qmel.DI_SEQUENCE_NUMBER AS qmel_seq,
    qmel.DI_OPERATION_TYPE AS qmel_operacao,
    qmel.MANDT AS qmel_cliente,
    qmel.QMNUM AS numero_notificacao,
    qmel.QMART AS tipo_notificacao,
    qmel.QMGRP AS grupo_qualidade,
    qmel.QMCOD AS codigo_qualidade,
    qmel.QMTXT AS texto_notificacao,
    qmel.QMDAB AS data_defeito,
    qmel.QMZEIT AS tempo_defeito,
    qmel.QMNGRP AS grupo_notificacao,
    qmel.QMREF AS referencia_notificacao,
    qmel.QMREF_OLD AS referencia_notificacao_antiga,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_2,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_2,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_3,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_3,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_4,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_4,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_5,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_5,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_6,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_6,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_7,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_7,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_8,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_8,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_9,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_9,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_10,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_10,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_11,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_11,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_12,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_12,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_13,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_13,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_14,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_14,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_15,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_15,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_16,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_16,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_17,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_17,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_18,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_18,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_19,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_19,
    qmel.QMREF_OLD_INT AS referencia_notificacao_antiga_int_20,
    qmel.QMREF_OLD_EXT AS referencia_notificacao_antiga_ext_20,
    qmel.dataImportacao AS qmel_data_importacao,
    
    -- CAMPOS DA TABELA RESB (Reservas)
    resb.DI_SEQUENCE_NUMBER AS resb_seq,
    resb.DI_OPERATION_TYPE AS resb_operacao,
    resb.MANDT AS resb_cliente,
    resb.RSNUM AS numero_roteiro_resb,
    resb.RSPOS AS posicao_roteiro,
    resb.RSART AS tipo_roteiro,
    resb.BDTER AS data_necessidade,
    resb.BDMNG AS quantidade_necessaria,
    resb.ENMNG AS quantidade_entrada,
    resb.ENTME AS unidade_medida_entrada,
    resb.AFMNG AS quantidade_af,
    resb.AFMEI AS unidade_medida_af,
    resb.WMENG AS quantidade_wm,
    resb.WMEIN AS unidade_medida_wm,
    resb.BDART AS tipo_necessidade,
    resb.BDBSA AS area_necessidade,
    resb.BDBSA AS area_necessidade_2,
    resb.BDBSA AS area_necessidade_3,
    resb.BDBSA AS area_necessidade_4,
    resb.BDBSA AS area_necessidade_5,
    resb.BDBSA AS area_necessidade_6,
    resb.BDBSA AS area_necessidade_7,
    resb.BDBSA AS area_necessidade_8,
    resb.BDBSA AS area_necessidade_9,
    resb.BDBSA AS area_necessidade_10,
    resb.BDBSA AS area_necessidade_11,
    resb.BDBSA AS area_necessidade_12,
    resb.BDBSA AS area_necessidade_13,
    resb.BDBSA AS area_necessidade_14,
    resb.BDBSA AS area_necessidade_15,
    resb.BDBSA AS area_necessidade_16,
    resb.BDBSA AS area_necessidade_17,
    resb.BDBSA AS area_necessidade_18,
    resb.BDBSA AS area_necessidade_19,
    resb.BDBSA AS area_necessidade_20,
    resb.dataImportacao AS resb_data_importacao,
    
    -- CAMPOS DA TABELA MSEG (Movimentações Material)
    mseg.DI_SEQUENCE_NUMBER AS mseg_seq,
    mseg.DI_OPERATION_TYPE AS mseg_operacao,
    mseg.MANDT AS mseg_cliente,
    mseg.MBLNR AS numero_documento_material,
    mseg.MJAHR AS ano_documento_material,
    mseg.ZEILE AS linha_documento_material,
    mseg.BWART AS tipo_movimentacao,
    mseg.BUDAT AS data_documento,
    mseg.CPUDT AS data_processamento,
    mseg.CPUTM AS tempo_processamento,
    mseg.USNAM AS nome_usuario,
    mseg.WERKS AS centro_trabalho_mseg,
    mseg.LGORT AS deposito_mseg,
    mseg.MATNR AS material_mseg,
    mseg.CHARG AS lote_mseg,
    mseg.MENGE AS quantidade_mseg,
    mseg.MEINS AS unidade_medida_mseg,
    mseg.BWART AS tipo_movimentacao_mseg,
    mseg.BUDAT AS data_documento_mseg,
    mseg.CPUDT AS data_processamento_mseg,
    mseg.CPUTM AS tempo_processamento_mseg,
    mseg.USNAM AS nome_usuario_mseg,
    mseg.WERKS AS centro_trabalho_mseg_2,
    mseg.LGORT AS deposito_mseg_2,
    mseg.MATNR AS material_mseg_2,
    mseg.CHARG AS lote_mseg_2,
    mseg.MENGE AS quantidade_mseg_2,
    mseg.MEINS AS unidade_medida_mseg_2,
    mseg.dataImportacao AS mseg_data_importacao,
    
    -- CAMPOS DA TABELA COEP (Itens Contabilidade)
    coep.DI_SEQUENCE_NUMBER AS coep_seq,
    coep.DI_OPERATION_TYPE AS coep_operacao,
    coep.MANDT AS coep_cliente,
    coep.MANDT AS coep_cliente_2,
    coep.MANDT AS coep_cliente_3,
    coep.MANDT AS coep_cliente_4,
    coep.MANDT AS coep_cliente_5,
    coep.MANDT AS coep_cliente_6,
    coep.MANDT AS coep_cliente_7,
    coep.MANDT AS coep_cliente_8,
    coep.MANDT AS coep_cliente_9,
    coep.MANDT AS coep_cliente_10,
    coep.MANDT AS coep_cliente_11,
    coep.MANDT AS coep_cliente_12,
    coep.MANDT AS coep_cliente_13,
    coep.MANDT AS coep_cliente_14,
    coep.MANDT AS coep_cliente_15,
    coep.MANDT AS coep_cliente_16,
    coep.MANDT AS coep_cliente_17,
    coep.MANDT AS coep_cliente_18,
    coep.MANDT AS coep_cliente_19,
    coep.MANDT AS coep_cliente_20,
    coep.dataImportacao AS coep_data_importacao,
    
    -- CAMPOS DA TABELA EKKN (Itens Conta Fornecedor)
    ekkn.DI_SEQUENCE_NUMBER AS ekkn_seq,
    ekkn.DI_OPERATION_TYPE AS ekkn_operacao,
    ekkn.MANDT AS ekkn_cliente,
    ekkn.EBELN AS numero_pedido_compra,
    ekkn.EBELP AS posicao_pedido_compra,
    ekkn.ZEKKN AS numero_item_conta_fornecedor,
    ekkn.VRTYP AS tipo_referencia,
    ekkn.VRSIO AS versao_referencia,
    ekkn.VRSIP AS posicao_referencia,
    ekkn.VRSIK AS chave_referencia,
    ekkn.VRSIZ AS item_referencia,
    ekkn.VRSIY AS ano_referencia,
    ekkn.VRSIM AS mes_referencia,
    ekkn.VRSID AS dia_referencia,
    ekkn.VRSIH AS hora_referencia,
    ekkn.VRSII AS minuto_referencia,
    ekkn.VRSIJ AS segundo_referencia,
    ekkn.VRSIK AS chave_referencia_2,
    ekkn.VRSIZ AS item_referencia_2,
    ekkn.VRSIY AS ano_referencia_2,
    ekkn.VRSIM AS mes_referencia_2,
    ekkn.VRSID AS dia_referencia_2,
    ekkn.VRSIH AS hora_referencia_2,
    ekkn.VRSII AS minuto_referencia_2,
    ekkn.VRSIJ AS segundo_referencia_2,
    ekkn.VRSIK AS chave_referencia_3,
    ekkn.VRSIZ AS item_referencia_3,
    ekkn.VRSIY AS ano_referencia_3,
    ekkn.VRSIM AS mes_referencia_3,
    ekkn.VRSID AS dia_referencia_3,
    ekkn.VRSIH AS hora_referencia_3,
    ekkn.VRSII AS minuto_referencia_3,
    ekkn.VRSIJ AS segundo_referencia_3,
    ekkn.dataImportacao AS ekkn_data_importacao,
    
    -- CAMPOS DA TABELA VBAK (Cabeçalho Vendas)
    vbak.DI_SEQUENCE_NUMBER AS vbak_seq,
    vbak.DI_OPERATION_TYPE AS vbak_operacao,
    vbak.MANDT AS vbak_cliente,
    vbak.VBELN AS numero_documento_venda,
    vbak.ERDAT AS data_criacao_venda,
    vbak.ERZET AS tempo_criacao_venda,
    vbak.ERNAM AS usuario_criacao_venda,
    vbak.ANGDT AS data_proposta,
    vbak.BNDDT AS data_vinculacao,
    vbak.AUDAT AS data_documento_venda,
    vbak.VBTYP AS tipo_documento_venda,
    vbak.TRVOG AS grupo_transacao_venda,
    vbak.AUART AS tipo_documento_venda_auart,
    vbak.AUGRU AS grupo_ordem_venda,
    vbak.GWLDT AS data_entrega,
    vbak.SUBMI AS subitem,
    vbak.LIFSK AS bloqueio_entrega,
    vbak.FAKSK AS bloqueio_faturamento,
    vbak.NETWR AS valor_liquido,
    vbak.WAERK AS moeda_venda,
    vbak.VKORG AS organizacao_venda,
    vbak.VTWEG AS canal_distribuicao,
    vbak.SPART AS divisao_venda,
    vbak.VKGRP AS grupo_venda,
    vbak.VKBUR AS escritorio_venda,
    vbak.GSBER AS area_negocio_venda,
    vbak.GSKST AS status_area_negocio,
    vbak.GUEBG AS data_inicio_validade,
    vbak.GUEEN AS data_fim_validade,
    vbak.KNUMV AS numero_condicao,
    vbak.DATAB AS data_inicio_validade_condicao,
    vbak.DATBI AS data_fim_validade_condicao,
    vbak.MAHNV AS indicador_aviso,
    vbak.KUNNR AS cliente_venda,
    vbak.LIFNR AS fornecedor_venda,
    vbak.KUNWE AS cliente_entrega,
    vbak.KUNRG AS cliente_faturamento,
    vbak.KUNRS AS cliente_remetente,
    vbak.KUNRS AS cliente_remetente_2,
    vbak.KUNRS AS cliente_remetente_3,
    vbak.KUNRS AS cliente_remetente_4,
    vbak.KUNRS AS cliente_remetente_5,
    vbak.KUNRS AS cliente_remetente_6,
    vbak.KUNRS AS cliente_remetente_7,
    vbak.KUNRS AS cliente_remetente_8,
    vbak.KUNRS AS cliente_remetente_9,
    vbak.KUNRS AS cliente_remetente_10,
    vbak.KUNRS AS cliente_remetente_11,
    vbak.KUNRS AS cliente_remetente_12,
    vbak.KUNRS AS cliente_remetente_13,
    vbak.KUNRS AS cliente_remetente_14,
    vbak.KUNRS AS cliente_remetente_15,
    vbak.KUNRS AS cliente_remetente_16,
    vbak.KUNRS AS cliente_remetente_17,
    vbak.KUNRS AS cliente_remetente_18,
    vbak.KUNRS AS cliente_remetente_19,
    vbak.KUNRS AS cliente_remetente_20,
    vbak.dataImportacao AS vbak_data_importacao,
    
    -- CAMPOS DA TABELA VBAP (Itens Vendas)
    vbap.DI_SEQUENCE_NUMBER AS vbap_seq,
    vbap.DI_OPERATION_TYPE AS vbap_operacao,
    vbap.MANDT AS vbap_cliente,
    vbap.VBELN AS numero_documento_venda_vbap,
    vbap.POSNR AS posicao_item_venda,
    vbap.MATNR AS material_vbap,
    vbap.MATWA AS material_venda,
    vbap.PMATN AS material_principal,
    vbap.CHARG AS lote_vbap,
    vbap.ABGRU AS motivo_rejeicao,
    vbap.ABGRU AS motivo_rejeicao_2,
    vbap.ABGRU AS motivo_rejeicao_3,
    vbap.ABGRU AS motivo_rejeicao_4,
    vbap.ABGRU AS motivo_rejeicao_5,
    vbap.ABGRU AS motivo_rejeicao_6,
    vbap.ABGRU AS motivo_rejeicao_7,
    vbap.ABGRU AS motivo_rejeicao_8,
    vbap.ABGRU AS motivo_rejeicao_9,
    vbap.ABGRU AS motivo_rejeicao_10,
    vbap.ABGRU AS motivo_rejeicao_11,
    vbap.ABGRU AS motivo_rejeicao_12,
    vbap.ABGRU AS motivo_rejeicao_13,
    vbap.ABGRU AS motivo_rejeicao_14,
    vbap.ABGRU AS motivo_rejeicao_15,
    vbap.ABGRU AS motivo_rejeicao_16,
    vbap.ABGRU AS motivo_rejeicao_17,
    vbap.ABGRU AS motivo_rejeicao_18,
    vbap.ABGRU AS motivo_rejeicao_19,
    vbap.ABGRU AS motivo_rejeicao_20,
    vbap.dataImportacao AS vbap_data_importacao

FROM poseidon_uc.saps4hanarawview.aufk AS aufk

-- LEFT JOIN com todas as tabelas complementares
LEFT JOIN poseidon_uc.saps4hanarawview.afko AS afko
    ON afko.MANDT = aufk.MANDT AND afko.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afih AS afih
    ON afih.MANDT = aufk.MANDT AND afih.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afpo AS afpo
    ON afpo.MANDT = aufk.MANDT AND afpo.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.afru AS afru
    ON afru.MANDT = aufk.MANDT AND afru.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.qmel AS qmel
    ON qmel.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.resb AS resb
    ON resb.MANDT = aufk.MANDT AND resb.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.mseg AS mseg
    ON mseg.MANDT = aufk.MANDT AND mseg.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.coep AS coep
    ON coep.MANDT = aufk.MANDT AND coep.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.ekkn AS ekkn
    ON ekkn.MANDT = aufk.MANDT AND ekkn.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.vbak AS vbak
    ON vbak.MANDT = aufk.MANDT AND vbak.AUFNR = aufk.AUFNR

LEFT JOIN poseidon_uc.saps4hanarawview.vbap AS vbap
    ON vbap.MANDT = aufk.MANDT AND vbap.AUFNR = aufk.AUFNR

WHERE
    aufk.AUART IN ('GIA','GAA','GRA') AND
    aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI') AND
    
    -- Mantém a sequência de alteração mais recente da ordem
    aufk.DI_SEQUENCE_NUMBER = (
        SELECT MAX(a.DI_SEQUENCE_NUMBER)
        FROM poseidon_uc.saps4hanarawview.aufk AS a
        WHERE a.AUFNR = aufk.AUFNR
    )

ORDER BY aufk.AUFNR